# **Importación de librerías** 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error
import math

# **PROCESAMIENTO DE DATOS**

Lectura de archivos

In [8]:
# Aprendizaje
disp_df = pd.read_csv("/content/drive/MyDrive/Aprendizaje Automático/Datos/disp_st40ns1.txt.bz2",
 compression="bz2",
 index_col=0)

# Test
comp_df = pd.read_csv("/content/drive/MyDrive/Aprendizaje Automático/Datos/comp_st40ns1.txt.bz2",
 compression="bz2",
 index_col=0)

FileNotFoundError: ignored

Partición de datos en conjuntos de entrenamiento y test

In [ ]:
train = disp_df.iloc[:5]
test = disp_df.iloc[3650:]

Randomización de las filas en los ficheros de train y test y valores de entrenamiento

In [ ]:
# Para la randomizacion de filas se usa la semilla 100429142
train_random = train.sample(frac=1, random_state=100429142).reset_index(drop=True)
test_random = test.sample(frac=1, random_state=100429142).reset_index(drop=True)

# Separa las variables independientes (X) de la variable dependiente (y)
X_train = train_random.iloc[:, :-1]
Y_train = train_random.iloc[:, -1]

# **MODELOS BÁSICOS**

## **OMISIÓN DE HIPERPARÁMETROS**

Modelo *KNN* con omisión de hiperparámetros

In [ ]:
# Crear el modelo con los parámetros por defecto
knn = KNeighborsRegressor()

# Entrena el modelo con el conjunto de entrenamiento
knn.fit(X_train, Y_train)

# Realiza predicciones en el conjunto de entrenamiento
y_pred_train = knn.predict(X_train)

# Calcula el MAE en el conjunto de entrenamiento
mae_train = mean_absolute_error(Y_train, y_pred_train)

# Calcula el RMSE en el conjunto de entrenamiento
rmse_train = math.sqrt(mean_squared_error(Y_train, y_pred_train))

# Muestra el valor del MAE y RMSE en el conjunto de entrenamiento
print('MAE en el conjunto de entrenamiento:', mae_train)
print('RMSE en el conjunto de entrenamiento:', rmse_train)

MAE en el conjunto de entrenamiento: 2320715.78630137
RMSE en el conjunto de entrenamiento: 3221546.9694545004


Modelo Árbol de decisión con omisión de hiperparámetros

In [ ]:
# Crear el modelo con los parámetros por defecto
tree = DecisionTreeRegressor()

# Entrenar el modelo
tree.fit(X_train, Y_train)

# Realizar predicciones en el conjunto de entrenamiento
Y_pred = tree.predict(X_train)

# Calcular el MAE y el RMSE
mae = mean_absolute_error(Y_train, Y_pred)
rmse = math.sqrt(mean_squared_error(Y_train, Y_pred))

# Imprimir los resultados
print("MAE: {:.2f}".format(mae))
print("RMSE: {:.2f}".format(rmse))

Modelo Árbol de regresión lineal con omisión de hiperparámetros

In [ ]:
# Crear modelo
modelo_rl = LinearRegression()

# Entrenar modelo
modelo_rl.fit(X_train, Y_train)

# Predecir valores para datos de test
y_pred = modelo_rl.predict(X_train)

mae = mean_absolute_error(Y_train, y_pred)
rmse = math.sqrt(mean_squared_error(Y_train, y_pred))

print("MAE:", mae)
print("RMSE:", rmse)

MAE: 2175824.6934650526
RMSE: 3137001.6072183605


## **OPTIMIZACIÓN DE HIPERPARÁMETROS**

Modelo KNN con hiperparámetros óptimos

In [ ]:
knn = KNeighborsRegressor()

# Definir cuadrícula de valores de hiperparámetros a explorar
param_grid = {'n_neighbors': [1, 3, 5, 7, 9], 'p': [1, 2]}

# Definir métricas de evaluación a utilizar
scoring = {'MAE': make_scorer(mean_absolute_error), 'RMSE': make_scorer(mean_squared_error, squared=False)}

# Realizar búsqueda exhaustiva de los mejores valores de los hiperparámetros
grid_search = GridSearchCV(knn, param_grid=param_grid, scoring=scoring, refit='RMSE', cv=5)
grid_search.fit(X_train, Y_train)

# Imprimir mejores valores de los hiperparámetros y su rendimiento en términos de MAE y RMSE
print('Mejores valores de hiperparámetros:', grid_search.best_params_)
print('MAE con hiperparámetros optimizados:', grid_search.cv_results_['mean_test_MAE'][grid_search.best_index_])
print('RMSE con hiperparámetros optimizados:', grid_search.cv_results_['mean_test_RMSE'][grid_search.best_index_])

Modelo Árbol de Decisión con hiperparámetros óptimos

In [ ]:
# Definir el modelo
tree = DecisionTreeRegressor()

# Definir los parámetros a probar
param_grid = {'max_depth': [3, 5, 7, 9], 'min_samples_leaf': [1, 2, 4, 8]}

# Definir la búsqueda por validación cruzada
grid_search = GridSearchCV(tree, param_grid, cv=5, scoring='neg_mean_squared_error')

# Entrenar el modelo con los datos de entrenamiento
grid_search.fit(X_train, Y_train)

# Obtener los mejores parámetros y el mejor modelo
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Realizar predicciones en el conjunto de entrenamiento
Y_pred = best_model.predict(X_train)

# Calcular el MAE y el RMSE
mae = mean_absolute_error(Y_train, Y_pred)
rmse = math.sqrt(mean_squared_error(Y_train, Y_pred))

# Imprimir los resultados
print("Mejores parámetros: ", best_params)
print("MAE: {:.2f}".format(mae))
print("RMSE: {:.2f}".format(rmse))

Modelo Árbol de Regresión Lineal con hiperparámetros óptimos

# **MODELOS AVANZADOS**

## **OMISIÓN DE HIPERPARÁMETROS**

Modelo SVMs con omisión de hiperparámetros

Modelo Random Forests con omisión de hiperparámetros

## **OPTIMIZACIÓN DE HIPERPARÁMETROS**

Modelo SVMs con hiperparámetros óptimos

Modelo Random Forests con hiperparámetros óptimos